## List of quality and tidiniess issues in soccer database

### Quality

#### Visual assessment

##### match table
- player positions (`home_player_X1` ... `away_player_X11`) are null for some matches
- player IDs (`home_player_1` ... `away_player_11`) are null for some matches
- match events (`goal` ... `possession`) are null for some matches
- predictions are null for some matches
- season should be categorical variable

##### player attributes table
- all attributes missing for some rows
- attacking work rate null for some players
- attacking work rate has strange values (`"None"`) for some players
- defensive work rate has strange values (`_0`, `o`) for some players
- attacking work rate should be categorical variable
- defensive work rate should be categorical variable
- attacking work rate has value `norm` which should be `medium` (or maybe it's a distinct level)
- attacking work rate has value `ormal` which should be `medium`
- attacking work rate has values `2`, `6`, `8`, `3`, `0`, `1`, `7`, `2` which should be mapped to the categorical values
- TODO: continue with defensive work rate

#### Programmatic assessment
- TODO: check is the associated columns in the matches table are null: league, season, date, match, home/away team

##### match table
TODOs
- get min and max dates to be sure they make sense
- get unique values in columns to see nonsensical values
- do any columns contain inconsistent data? eg mixed date formats, mixed categories

### Tidiness